In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import random
import matplotlib
import matplotlib.pyplot as plt
import time
import random

from sklearn.metrics import confusion_matrix, plot_confusion_matrix, accuracy_score, plot_roc_curve,\
                             precision_recall_curve, plot_precision_recall_curve, f1_score, average_precision_score,\
                             hinge_loss, precision_score, recall_score, classification_report
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, KFold
from sklearn.metrics import make_scorer, accuracy_score, average_precision_score, f1_score,\
                            log_loss, precision_score, recall_score, roc_auc_score
from sklearn.preprocessing import label_binarize, LabelBinarizer, LabelEncoder, OneHotEncoder
from sklearn.datasets import fetch_openml
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.utils import resample

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import RMSprop
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import to_categorical

from catboost import CatBoostClassifier

Using TensorFlow backend.
C:\Users\Andrzej\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Andrzej\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Andrzej\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Andrzej\anaconda3\lib\site-packages\tensorflow\python\framework\d

# Data load

In [2]:
fmnist = fetch_openml("Fashion-MNIST", data_home="./fmnist", cache=True)
classes = [str(x) for x in range(0, 10)]
num_classes = len(classes)

def mk_dataset(total, fmnist=fmnist, classes=classes):
    samples = int(fmnist.data.shape[0]*total)
    return resample(fmnist.data, fmnist.target, n_samples=samples)

def plot_imgs(x, y, w=28, h=28):
    plt.figure(figsize=(10,10))
    for i in range(min(25, x.shape[0])):
        plt.subplot(5,5,i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        img = x[i]
        img = img.reshape((w, h))
        plt.imshow(img)
        plt.xlabel(y[i])
    plt.show()    

# Model evaluation

In [18]:
def binarized_scorer(metric, **kwargs):
    lb = LabelBinarizer()
    def score(y_test, y_pred, metric=metric, lb=lb, kwargs=kwargs):
        lb.fit(y_test)
        y_test = lb.transform(y_test)
        y_pred = lb.transform(y_pred)
        return metric(y_test, y_pred, **kwargs)
    return score

def mk_test(clf, name, mangling=0.0):
    def run_test(X, Y, clf=clf, name=name, mangling=mangling):
        scoring = {
            "accuracy":     binarized_scorer(accuracy_score), 
            "f1_score":     binarized_scorer(f1_score, average='macro'), 
            "log_loss":     binarized_scorer(log_loss), 
            "precision":    binarized_scorer(precision_score, average='macro'), 
            "recall":       binarized_scorer(recall_score, average='macro'), 
            "roc_auc":      binarized_scorer(roc_auc_score, average='macro'),
            # to je pole pod Precision-Recall, albo jakaś średnia. nie wiem.
            "prec_rec_auc": binarized_scorer(average_precision_score, average='macro') 
        }
        
        # Ta implementacja jest potwornie wolna dla modeli szkolonych na CPU
        # ale przynajmniej pozwala na wprowadzenie szumu do danych treningowych
        scores = {}
        Y_org = np.array(Y, copy=True)
        for train_index, test_index in KFold(n_splits=5).split(X):
            Y = np.array(Y_org, copy=True)
            to_mangle = int(len(train_index)*mangling)
            for idx in train_index[:to_mangle]:
                Y[idx] = random.choice(classes)
            clf.fit(X[train_index], Y[train_index])
            Y_true = Y[test_index]
            Y_pred = clf.predict(X[test_index])
            for (key, scorer) in scoring.items():
                if key in scores:
                    scores[key] += [scorer(Y_true, Y_pred)]
                else:
                    scores[key] = [scorer(Y_true, Y_pred)]
        scores = {k: (1.0*sum(v))/len(v) for k, v in scores.items()}
        df = pd.DataFrame(scores, index=[0])
        df.insert(loc=0, column='Name', value=name)
        return df
    return run_test

# Models

In [4]:
def mk_adaboost(depth=5, n=100, seed=1):
    return AdaBoostClassifier(
        base_estimator=DecisionTreeClassifier(max_depth=depth),
        n_estimators=n,
        random_state=seed)

def mk_catboost():
    return CatBoostClassifier(iterations=1000, task_type="GPU")

class MyLittleKerasClassifier(KerasClassifier):
    # predict() nie zwracal 
    def predict(self, X):
        y_pred = KerasClassifier.predict(self, X)
        return to_categorical(y_pred, num_classes)

def mk_mlp(epochs=10):
    def build():
        model = Sequential()
        # starannie dobrane wartosci, wiem co robie
        model.add(Dense(256, activation='relu', input_shape=(28*28,)))
        model.add(Dropout(0.2))
        model.add(Dense(512, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(512, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(512, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(256, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(num_classes, activation='softmax'))
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        return model
    return MyLittleKerasClassifier(build_fn=build, epochs=epochs)

def mk_cnn(epochs=10):
    def build():
        model = Sequential()
        model.add(Conv2D(filters=32, kernel_size=(3, 3), input_shape=(28,28, 1), activation='relu'))
        model.add(BatchNormalization())
        model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
        model.add(BatchNormalization())
        model.add(Conv2D(32, kernel_size = 5, strides=2, padding='same', activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))
        model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
        model.add(BatchNormalization())
        model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
        model.add(BatchNormalization())
        model.add(Conv2D(64, kernel_size = 5, strides=2, padding='same', activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))
        model.add(Conv2D(128, kernel_size = 4, activation='relu'))
        model.add(BatchNormalization())
        model.add(Flatten())
        model.add(Dropout(0.4))
        model.add(Dense(10, activation='softmax'))
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        return model
    return MyLittleKerasClassifier(build_fn=build, epochs=epochs)

In [23]:
def test_cnn(data_sz, epochs):
    name = 'CNN | {}% of data'.format(data_sz*100.0)
    name += ' | {} epochs'.format(epochs) if epochs else ''
    x, y = mk_dataset(data_sz)
    y = to_categorical(y, num_classes)
    x /= 255.0
    x = x.reshape((x.shape[0], 28, 28, 1))
    return mk_test(mk_cnn(epochs=epochs), name)(x, y)

def test_mlp(data_sz, epochs, mangling=0.0):
    name = 'MLP | {}% of data'.format(data_sz*100.0)
    name += ' | {} epochs'.format(epochs) if epochs else ''
    x, y = mk_dataset(data_sz)
    y = to_categorical(y, num_classes)
    x /= 255.0
    return mk_test(mk_mlp(epochs=epochs), name)(x, y)

# Pomiar z użyciem time.time() to nie jest czas CPU
# ale już tam nic to. timeit() było zbyt irytujące
def test_adaboost(data_sz, epochs=100, duration=False):
    name = 'AdaBoost | {}% of data'.format(data_sz*100.0)
    start_time = time.time()
    result = mk_test(mk_adaboost(n=epochs), name)(*mk_dataset(data_sz))
    elapsed = time.time()-start_time
    if duration:
        result['Name'] += ' | %d secs' % elapsed
    result['Name'] += ' | {}% of train set mangled'.format(mangling*100) if mangling > 0.0 else ''
    return result

# Pomiar metryk w zależności od ilości dostępnych danych

In [ ]:
tests_data = pd.concat([method(data_sz, None) for method in [test_adaboost, test_mlp, test_cnn] for data_sz in [0.1, 0.5, 1.0]])
tests_data

# Pomiar metryk w zależności od czasów treningu

## Dla Adaboosta - czas rzeczywisty

In [51]:
tests_time = [test_adaboost(0.5, n=n, duration=True) for n in [50, 150, 500]]
pd.concat(tests_time)

,Name,accuracy,f1_score,log_loss,precision,recall,roc_auc,prec_rec_auc
0,AdaBoost | 50.0% of data | 212 secs,0.688314,0.688106,10.765243,0.697314,0.688081,0.826725,0.550447
0,AdaBoost | 50.0% of data | 627 secs,0.753543,0.752674,8.512328,0.755930,0.753249,0.862933,0.628641
0,AdaBoost | 50.0% of data | 1785 secs,0.789543,0.789982,7.268932,0.792207,0.790228,0.883419,0.672362


## Dla sieci - liczba epok

In [54]:
tests_time += [method(0.5, epochs) for method in [test_mlp, test_cnn] for epochs in [5, 15, 25]]
tests_time = pd.concat(tests_time)
tests_time.to_pickle("3b.pkl")
tests_time

Epoch 1/5
28000/28000 [==============================] - 3s 102us/step - loss: 0.1148 - accuracy: 0.9544
Epoch 2/5
28000/28000 [==============================] - 3s 95us/step - loss: 0.0879 - accuracy: 0.9656
Epoch 3/5
28000/28000 [==============================] - 3s 98us/step - loss: 0.0799 - accuracy: 0.9685
Epoch 4/5
28000/28000 [==============================] - 3s 91us/step - loss: 0.0765 - accuracy: 0.9698
Epoch 5/5
28000/28000 [==============================] - 3s 96us/step - loss: 0.0721 - accuracy: 0.9717
Epoch 1/5
28000/28000 [==============================] - 3s 106us/step - loss: 0.1150 - accuracy: 0.9539
Epoch 2/5
28000/28000 [==============================] - 3s 92us/step - loss: 0.0873 - accuracy: 0.9657
Epoch 3/5
28000/28000 [==============================] - 3s 92us/step - loss: 0.0804 - accuracy: 0.9688
Epoch 4/5
28000/28000 [==============================] - 3s 92us/step - loss: 0.0755 - accuracy: 0.9706
Epoch 5/5
28000/28000 [==============================] - 3s 91

28000/28000 [==============================] - 3s 100us/step - loss: 0.0739 - accuracy: 0.9708
Epoch 6/25
28000/28000 [==============================] - 3s 99us/step - loss: 0.0706 - accuracy: 0.9723
Epoch 7/25
28000/28000 [==============================] - 3s 100us/step - loss: 0.0691 - accuracy: 0.9728
Epoch 8/25
28000/28000 [==============================] - 3s 101us/step - loss: 0.0668 - accuracy: 0.9737
Epoch 9/25
28000/28000 [==============================] - 3s 101us/step - loss: 0.0660 - accuracy: 0.9739
Epoch 10/25
28000/28000 [==============================] - 3s 102us/step - loss: 0.0635 - accuracy: 0.9746
Epoch 11/25
28000/28000 [==============================] - 3s 99us/step - loss: 0.0622 - accuracy: 0.9756
Epoch 12/25
28000/28000 [==============================] - 3s 99us/step - loss: 0.0609 - accuracy: 0.9756
Epoch 13/25
28000/28000 [==============================] - 3s 99us/step - loss: 0.0610 - accuracy: 0.9761
Epoch 14/25
28000/28000 [==============================] 

28000/28000 [==============================] - 7s 234us/step - loss: 0.0787 - accuracy: 0.9691
Epoch 3/5
28000/28000 [==============================] - 7s 248us/step - loss: 0.0663 - accuracy: 0.9739
Epoch 4/5
28000/28000 [==============================] - 7s 246us/step - loss: 0.0599 - accuracy: 0.9764
Epoch 5/5
28000/28000 [==============================] - 7s 236us/step - loss: 0.0553 - accuracy: 0.9786
Epoch 1/5
28000/28000 [==============================] - 8s 277us/step - loss: 0.1193 - accuracy: 0.9535
Epoch 2/5
28000/28000 [==============================] - 7s 253us/step - loss: 0.0790 - accuracy: 0.9684
Epoch 3/5
28000/28000 [==============================] - 7s 267us/step - loss: 0.0673 - accuracy: 0.9739
Epoch 4/5
28000/28000 [==============================] - 7s 251us/step - loss: 0.0595 - accuracy: 0.9766
Epoch 5/5
28000/28000 [==============================] - 7s 238us/step - loss: 0.0560 - accuracy: 0.9782
Epoch 1/5
28000/28000 [==============================] - 8s 288us

Epoch 11/25
28000/28000 [==============================] - 9s 329us/step - loss: 0.0376 - accuracy: 0.9851
Epoch 12/25
28000/28000 [==============================] - 9s 321us/step - loss: 0.0357 - accuracy: 0.9861
Epoch 13/25
28000/28000 [==============================] - 9s 329us/step - loss: 0.0323 - accuracy: 0.9876
Epoch 14/25
28000/28000 [==============================] - 9s 333us/step - loss: 0.0322 - accuracy: 0.9878
Epoch 15/25
28000/28000 [==============================] - 9s 327us/step - loss: 0.0296 - accuracy: 0.9890
Epoch 16/25
28000/28000 [==============================] - 9s 326us/step - loss: 0.0282 - accuracy: 0.9894
Epoch 17/25
28000/28000 [==============================] - 9s 326us/step - loss: 0.0267 - accuracy: 0.9898
Epoch 18/25
28000/28000 [==============================] - 38s 1ms/step - loss: 0.0258 - accuracy: 0.9904
Epoch 19/25
28000/28000 [==============================] - 16s 568us/step - loss: 0.0243 - accuracy: 0.9911
Epoch 20/25
28000/28000 [============

,Name,accuracy,f1_score,log_loss,precision,recall,roc_auc,prec_rec_auc
0,AdaBoost | 50.0% of data | 212 secs,0.688314,0.688106,10.765243,0.697314,0.688081,0.826725,0.550447
0,AdaBoost | 50.0% of data | 627 secs,0.753543,0.752674,8.512328,0.755930,0.753249,0.862933,0.628641
0,AdaBoost | 50.0% of data | 1785 secs,0.789543,0.789982,7.268932,0.792207,0.790228,0.883419,0.672362
0,MLP | 50.0% of data| 5 epochs,0.858800,0.858106,4.876875,0.863362,0.858452,0.921385,0.764586
0,MLP | 50.0% of data| 15 epochs,0.882400,0.880851,4.061760,0.882991,0.881708,0.934326,0.798184
0,MLP | 50.0% of data| 25 epochs,0.885657,0.884865,3.949262,0.887206,0.885287,0.936292,0.804582
0,CNN | 50.0% of data| 5 epochs,0.896600,0.896143,3.571309,0.899990,0.896879,0.942695,0.823182
0,CNN | 50.0% of data| 15 epochs,0.928200,0.928281,2.479884,0.930066,0.928377,0.960202,0.873302
0,CNN | 50.0% of data| 25 epochs,0.943486,0.943669,1.951934,0.944031,0.943767,0.968742,0.898537


# Moc klasyfiaktora
Tutaj trzeba przemieszać pewien % etykiet zbioru treningowego.

In [28]:
def rob_test_cnn(mangling, data_sz=0.5, epochs=10):
    name = 'CNN | {}% of data | {}% mangling'.format(data_sz*100.0, mangling*100.0)
    x, y = mk_dataset(data_sz)
    y = to_categorical(y, num_classes)
    x /= 255.0
    x = x.reshape((x.shape[0], 28, 28, 1))
    return mk_test(mk_cnn(epochs=epochs), name, mangling=mangling)(x, y)

def rob_test_mlp(mangling, data_sz=0.5, epochs=10):
    name = 'MLP | {}% of data | {}% mangling'.format(data_sz*100.0, mangling*100.0)
    x, y = mk_dataset(data_sz)
    y = to_categorical(y, num_classes)
    x /= 255.0
    return mk_test(mk_mlp(epochs=epochs), name, mangling=mangling)(x, y)

def rob_test_adaboost(mangling, data_sz=0.5, n=100):
    name = 'AdaBoost | {}% of data | {}% mangling'.format(data_sz*100.0, mangling*100.0)
    result = mk_test(mk_adaboost(n=n), name, mangling=mangling)(*mk_dataset(data_sz))
    return result

In [31]:
robustness_test = [method(man) for method in [rob_test_adaboost, rob_test_mlp, rob_test_cnn] for man in [0.01, 0.05, 0.15, 0.3]]
robustness_test

Epoch 1/10
28000/28000 [==============================] - 8s 277us/step - loss: 0.3792 - accuracy: 0.9106
Epoch 2/10
28000/28000 [==============================] - 6s 220us/step - loss: 0.3669 - accuracy: 0.9109
Epoch 3/10
28000/28000 [==============================] - 6s 220us/step - loss: 0.3643 - accuracy: 0.9106
Epoch 4/10
28000/28000 [==============================] - 6s 223us/step - loss: 0.3752 - accuracy: 0.9094
Epoch 5/10
28000/28000 [==============================] - 6s 220us/step - loss: 0.3525 - accuracy: 0.9157
Epoch 6/10
28000/28000 [==============================] - 6s 223us/step - loss: 0.3499 - accuracy: 0.9171
Epoch 7/10
28000/28000 [==============================] - 6s 221us/step - loss: 0.3487 - accuracy: 0.91750s -
Epoch 8/10
28000/28000 [==============================] - 6s 220us/step - loss: 0.3355 - accuracy: 0.9237
Epoch 9/10
28000/28000 [==============================] - 6s 224us/step - loss: 0.3323 - accuracy: 0.9227
Epoch 10/10
28000/28000 [=================

28000/28000 [==============================] - 7s 260us/step - loss: 0.7573 - accuracy: 0.8687
Epoch 9/10
28000/28000 [==============================] - 7s 259us/step - loss: 0.7612 - accuracy: 0.8670
Epoch 10/10
28000/28000 [==============================] - 7s 262us/step - loss: 0.7580 - accuracy: 0.8676
Epoch 1/10
28000/28000 [==============================] - 9s 332us/step - loss: 0.8105 - accuracy: 0.8630
Epoch 2/10
28000/28000 [==============================] - 7s 265us/step - loss: 0.8079 - accuracy: 0.8625
Epoch 3/10
28000/28000 [==============================] - 7s 268us/step - loss: 0.8061 - accuracy: 0.8627
Epoch 4/10
28000/28000 [==============================] - 7s 248us/step - loss: 0.8036 - accuracy: 0.8633
Epoch 5/10
28000/28000 [==============================] - 7s 246us/step - loss: 0.7957 - accuracy: 0.8642
Epoch 6/10
28000/28000 [==============================] - 7s 251us/step - loss: 0.7962 - accuracy: 0.8641
Epoch 7/10
28000/28000 [==============================] 

28000/28000 [==============================] - 8s 270us/step - loss: 3.2153 - accuracy: 0.6611
Epoch 6/10
28000/28000 [==============================] - 7s 266us/step - loss: 3.2155 - accuracy: 0.6612
Epoch 7/10
28000/28000 [==============================] - 8s 270us/step - loss: 3.2156 - accuracy: 0.6612
Epoch 8/10
28000/28000 [==============================] - 7s 267us/step - loss: 3.2153 - accuracy: 0.6612
Epoch 9/10
28000/28000 [==============================] - 8s 270us/step - loss: 3.2146 - accuracy: 0.6611
Epoch 10/10
28000/28000 [==============================] - 7s 266us/step - loss: 3.2146 - accuracy: 0.6612
Epoch 1/10
28000/28000 [==============================] - 9s 336us/step - loss: 3.2461 - accuracy: 0.6585
Epoch 2/10
28000/28000 [==============================] - 7s 267us/step - loss: 3.2422 - accuracy: 0.6585
Epoch 3/10
28000/28000 [==============================] - 8s 271us/step - loss: 3.2410 - accuracy: 0.6586
Epoch 4/10
28000/28000 [==============================] 

Epoch 2/10
28000/28000 [==============================] - 13s 456us/step - loss: 0.2870 - accuracy: 0.9364
Epoch 3/10
28000/28000 [==============================] - 13s 457us/step - loss: 0.2696 - accuracy: 0.9406
Epoch 4/10
28000/28000 [==============================] - 13s 458us/step - loss: 0.2606 - accuracy: 0.9439
Epoch 5/10
28000/28000 [==============================] - 13s 457us/step - loss: 0.2613 - accuracy: 0.9433
Epoch 6/10
28000/28000 [==============================] - 13s 457us/step - loss: 0.2476 - accuracy: 0.9480
Epoch 7/10
28000/28000 [==============================] - 13s 457us/step - loss: 0.2528 - accuracy: 0.9481
Epoch 8/10
28000/28000 [==============================] - 13s 457us/step - loss: 0.2414 - accuracy: 0.9503
Epoch 9/10
28000/28000 [==============================] - 13s 458us/step - loss: 0.2376 - accuracy: 0.9524
Epoch 10/10
28000/28000 [==============================] - 13s 456us/step - loss: 0.2319 - accuracy: 0.9539
Epoch 1/10
28000/28000 [============

28000/28000 [==============================] - 14s 517us/step - loss: 1.7365 - accuracy: 0.7802
Epoch 9/10
28000/28000 [==============================] - 14s 516us/step - loss: 1.7369 - accuracy: 0.7801
Epoch 10/10
28000/28000 [==============================] - 14s 514us/step - loss: 1.7359 - accuracy: 0.7801
Epoch 1/10
28000/28000 [==============================] - 18s 655us/step - loss: 1.9906 - accuracy: 0.7767
Epoch 2/10
28000/28000 [==============================] - 15s 519us/step - loss: 1.8326 - accuracy: 0.7789
Epoch 3/10
28000/28000 [==============================] - 15s 520us/step - loss: 1.7884 - accuracy: 0.7796
Epoch 4/10
28000/28000 [==============================] - 15s 521us/step - loss: 1.7621 - accuracy: 0.7801
Epoch 5/10
28000/28000 [==============================] - 15s 519us/step - loss: 1.7490 - accuracy: 0.7804
Epoch 6/10
28000/28000 [==============================] - 15s 520us/step - loss: 1.7431 - accuracy: 0.7803
Epoch 7/10
28000/28000 [=======================

28000/28000 [==============================] - 16s 585us/step - loss: 3.2190 - accuracy: 0.6610
Epoch 5/10
28000/28000 [==============================] - 16s 559us/step - loss: 3.2147 - accuracy: 0.6610
Epoch 6/10
28000/28000 [==============================] - 16s 579us/step - loss: 3.2138 - accuracy: 0.6610
Epoch 7/10
28000/28000 [==============================] - 16s 577us/step - loss: 3.2136 - accuracy: 0.6610
Epoch 8/10
28000/28000 [==============================] - 16s 559us/step - loss: 3.2132 - accuracy: 0.6610
Epoch 9/10
28000/28000 [==============================] - 16s 560us/step - loss: 3.2128 - accuracy: 0.6610
Epoch 10/10
28000/28000 [==============================] - 16s 561us/step - loss: 3.2123 - accuracy: 0.6610
Epoch 1/10
28000/28000 [==============================] - 21s 735us/step - loss: 3.6373 - accuracy: 0.6579
Epoch 2/10
28000/28000 [==============================] - 16s 563us/step - loss: 3.2867 - accuracy: 0.6607
Epoch 3/10
28000/28000 [=======================

[                                       Name  accuracy  f1_score  log_loss  \
 0  AdaBoost | 50.0% of data | 1.0% mangling  0.730743  0.736118  9.299812   
 
    precision    recall   roc_auc  prec_rec_auc  
 0   0.746584  0.731905  0.850987      0.600884  ,
                                        Name  accuracy  f1_score  log_loss  \
 0  AdaBoost | 50.0% of data | 5.0% mangling  0.718343  0.724324  9.728093   
 
    precision    recall   roc_auc  prec_rec_auc  
 0   0.738498  0.718751  0.843729      0.581152  ,
                                         Name  accuracy  f1_score   log_loss  \
 0  AdaBoost | 50.0% of data | 15.0% mangling  0.671886  0.674869  11.332666   
 
    precision    recall   roc_auc  prec_rec_auc  
 0   0.684017  0.670356  0.816965      0.512118  ,
                                         Name  accuracy  f1_score   log_loss  \
 0  AdaBoost | 50.0% of data | 30.0% mangling  0.639057  0.639461  12.466525   
 
    precision    recall   roc_auc  prec_rec_auc  
 0    0

In [34]:
df = pd.concat(robustness_test)
df.to_pickle('4.pkl')
df

,Name,accuracy,f1_score,log_loss,precision,recall,roc_auc,prec_rec_auc
0,AdaBoost | 50.0% of data | 1.0% mangling,0.730743,0.736118,9.299812,0.746584,0.731905,0.850987,0.600884
0,AdaBoost | 50.0% of data | 5.0% mangling,0.718343,0.724324,9.728093,0.738498,0.718751,0.843729,0.581152
0,AdaBoost | 50.0% of data | 15.0% mangling,0.671886,0.674869,11.332666,0.684017,0.670356,0.816965,0.512118
0,AdaBoost | 50.0% of data | 30.0% mangling,0.639057,0.639461,12.466525,0.643230,0.637984,0.798946,0.461308
0,MLP | 50.0% of data | 1.0% mangling,0.740486,0.725875,8.963306,0.742222,0.740946,0.856061,0.602340
0,MLP | 50.0% of data | 5.0% mangling,0.671771,0.652288,11.336613,0.715568,0.671952,0.817741,0.536828
0,MLP | 50.0% of data | 15.0% mangling,0.771686,0.754014,7.885696,0.781151,0.771444,0.873045,0.646986
0,MLP | 50.0% of data | 30.0% mangling,0.796657,0.782076,7.023213,0.796691,0.795308,0.886360,0.677405
0,CNN | 50.0% of data | 1.0% mangling,0.843114,0.838424,5.418641,0.843457,0.842359,0.912466,0.737045
0,CNN | 50.0% of data | 5.0% mangling,0.814286,0.809099,6.414344,0.823226,0.815660,0.897520,0.701925
